# Bootstrap Quantification under Prior Shift

In [ ]:
import pandas as pd

from pyquantification.experiments import cached_experiments

In [ ]:
quantification_methods = [
    'em-bs', 
]
dataset_labels = {
    'insect-sex': 'ISX',
    'insect-species': 'ISP',
    'arabic-digits': 'DIG',
    'handwritten-letters-letter': 'HLL',
    'handwritten-letters-author': 'HLA',
}
results_df = cached_experiments(
    cache_key='bootstrap_prior_shift_results',
    dataset_names=list(dataset_labels.keys()),
    classifier_names=['logreg'],
    calibration_methods=['uncalibrated'],
    loss_weights=[0],
    gain_weights=[0],
    random_states=list(range(0, 1000)),
    shift_types=['prior_shift'],
    bin_counts=['auto'],
    random_priors_options=[True],
    quantification_methods=quantification_methods,
    classification_workers=10,
    continue_on_failure=True,
)

## Quantification Method Comparison

In [ ]:
prior_df = results_df

def print_table_latex(table_df):
    for _, row in table_df.iterrows():
        if row.isna().all():
            print('\hline')
        else:
            print(' & '.join(row.to_dict().values()) + r' \\')

def coverage_table():
    experiment_grouping = ['dataset_name', 'shift_type', 'gain_weight',
                           'loss_weight', 'random_state']
    plot_methods = {
        'em-bs': 'EM-BS',
    }

    def format_cell(mean):
        str_mean = f'{mean:.0%}'
        str_mean = str_mean.replace('%', '\%')
        if mean >= 0.8:
            str_mean = r'\textbf{' + str_mean + '}'
        return str_mean

    rows = []
    for dataset_name, dataset_label in dataset_labels.items():
        for method, method_label in plot_methods.items():
            row = {
                'dataset': dataset_label,
                'method': method_label,
            }
            cell_prior_df = prior_df[prior_df['dataset_name'] == dataset_name]
            row['prior_shift'] = format_cell(cell_prior_df[f'{method}_coverage'].mean())
            rows.append(row)
        rows.append({})
    return pd.DataFrame(rows)
    
table_df = coverage_table()
display(table_df)
print_table_latex(table_df)